# Applied AI
## Homework 1 
### Ghazal Alinezhad Noghre 


### Data Manupilation

First we have to import libraries that we need:

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
import datetime
import cv2
from tqdm import tqdm

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1], 'GPU')

In the cell below, I am getting a list of files that exist in the dataset and create a pandas dataframe. Then I split them to training and validation set with the ratio of 0.2. 

In [3]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

data_dir = "/home/galinezh/hw/ai/hw1/train/"
file_names = os.listdir(data_dir)
print('Total number of images: {}'.format(len(file_names)))

files, labels = list(), list()
for file in file_names:
    files.append(file)
    labels.append(file[:3])
df = pd.DataFrame({'filename':files, 'label':labels})

train_set, valid_set = train_test_split(df, test_size=0.2,random_state=seed)
print('Number of images in training set: {}'.format(train_set.shape[0]))
print('Number of images in validation set: {}'.format(valid_set.shape[0]))

df.head()

Total number of images: 25000
Number of images in training set: 20000
Number of images in validation set: 5000


,filename,label
0,cat.797.jpg,cat
1,dog.5704.jpg,dog
2,cat.10234.jpg,cat
3,dog.6027.jpg,dog
4,cat.7413.jpg,cat


In the next step, I normalize and augment the data using ImageDataGenerator. I use rescale for normalizing the pixel values (dividing by 255 to map the value range to 0 and 1). For augmentation I use zoom, horiontal flip, shifts and shear. keep in mind that the augmentation is only needed for training set, so I will have different generators for training and validation set. Also, I batch the data with the batch size of 128.

In [4]:
image_size = (150, 150)
batch_size = 128

train_datagenerator = ImageDataGenerator(rotation_range=20, 
                                  rescale=1./255, 
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

valid_datagenerator = ImageDataGenerator(rescale=1./255)


training_data = train_datagenerator.flow_from_dataframe(dataframe=train_set, 
                                                        directory='./train',
                                                       x_col='filename',
                                                       y_col='label',
                                                       target_size=image_size,
                                                       class_mode='categorical',
                                                       batch_size=batch_size)

validation_data = valid_datagenerator.flow_from_dataframe(dataframe=valid_set,
                                                         directory='./train',
                                                         x_col='filename',
                                                         y_col='label',
                                                         target_size=image_size,
                                                         class_mode='categorical',
                                                         batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


Next, I use instatiate models and load them using pretrained weights. I will not use the final layers (classifier layers). Instead I add classification layer with Leaky Relu activation function and then finetune the model. The final layer has 2 neurons (representing the probablity of belonging to cat or dog class).

### VGG16

In [37]:
input_size = (150, 150 ,3)

VGG_model = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_shape=input_size)
VGG_model.trainable = False


model_VGG = keras.models.Sequential()
model_VGG.add(VGG_model)
model_VGG.add(Flatten())
model_VGG.add(Dense(128, activation=tf.nn.leaky_relu))
model_VGG.add(Dropout(0.3))
model_VGG.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [38]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_VGG.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/VGG/" + "VGG_transfer"
tensorboard_callback_VGG = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_VGG.fit(training_data,
            epochs=5,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_VGG] 
            )

Epoch 1/5
156/156 [==============================] - 135s 863ms/step - loss: 0.7735 - accuracy: 0.8275 - val_loss: 0.2414 - val_accuracy: 0.8962
Epoch 2/5
156/156 [==============================] - 135s 868ms/step - loss: 0.2998 - accuracy: 0.8697 - val_loss: 0.2257 - val_accuracy: 0.9079
Epoch 3/5
156/156 [==============================] - 136s 874ms/step - loss: 0.2736 - accuracy: 0.8829 - val_loss: 0.2910 - val_accuracy: 0.8718
Epoch 4/5
156/156 [==============================] - 136s 873ms/step - loss: 0.2885 - accuracy: 0.8777 - val_loss: 0.2043 - val_accuracy: 0.9161
Epoch 5/5
156/156 [==============================] - 135s 867ms/step - loss: 0.3139 - accuracy: 0.8705 - val_loss: 0.2020 - val_accuracy: 0.9189


For fine tuning I unfreez the backbone, and then finetune the whole model. Also, I use a smaller learning rate since I am training the whole model.

In [39]:
model_VGG.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_VGG.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/VGG/" + "VGG_finetune"
tensorboard_callback_VGG = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_VGG.fit(training_data,
            epochs=25,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_VGG] 
            )

Epoch 1/25
156/156 [==============================] - 137s 878ms/step - loss: 28.9374 - accuracy: 0.5089 - val_loss: 0.6915 - val_accuracy: 0.5016
Epoch 2/25
156/156 [==============================] - 138s 884ms/step - loss: 0.7749 - accuracy: 0.4988 - val_loss: 0.6962 - val_accuracy: 0.4998
Epoch 3/25
156/156 [==============================] - 140s 896ms/step - loss: 0.6886 - accuracy: 0.5429 - val_loss: 0.6659 - val_accuracy: 0.5917
Epoch 4/25
156/156 [==============================] - 139s 893ms/step - loss: 0.6796 - accuracy: 0.5587 - val_loss: 0.6413 - val_accuracy: 0.6340
Epoch 5/25
156/156 [==============================] - 137s 877ms/step - loss: 0.6462 - accuracy: 0.6218 - val_loss: 0.6176 - val_accuracy: 0.6815
Epoch 6/25
156/156 [==============================] - 139s 888ms/step - loss: 0.6073 - accuracy: 0.6658 - val_loss: 0.5601 - val_accuracy: 0.7161
Epoch 7/25
156/156 [==============================] - 138s 886ms/step - loss: 0.5637 - accuracy: 0.7101 - val_loss: 0.5141 

### MobileNetV2

In [14]:
input_size = (150, 150 ,3)

MobileNetV2_model = keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=input_size)
MobileNetV2_model.trainable = False


model_MobileNetV2 = keras.models.Sequential()
model_MobileNetV2.add(MobileNetV2_model)
model_MobileNetV2.add(Flatten())
model_MobileNetV2.add(Dense(128, activation=tf.nn.leaky_relu))
model_MobileNetV2.add(Dropout(0.3))
model_MobileNetV2.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [15]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_MobileNetV2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/MobileNetV2/" + "MobileNetV2_transfer"
tensorboard_callback_MobileNetV2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_MobileNetV2.fit(training_data,
            epochs=5,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_MobileNetV2] 
            )

Epoch 1/5
156/156 [==============================] - 135s 866ms/step - loss: 12.7261 - accuracy: 0.9054 - val_loss: 0.5631 - val_accuracy: 0.9621
Epoch 2/5
156/156 [==============================] - 133s 855ms/step - loss: 1.2644 - accuracy: 0.9285 - val_loss: 1.2925 - val_accuracy: 0.9631
Epoch 3/5
156/156 [==============================] - 133s 851ms/step - loss: 0.9516 - accuracy: 0.9305 - val_loss: 1.0261 - val_accuracy: 0.9555
Epoch 4/5
156/156 [==============================] - 133s 850ms/step - loss: 0.7977 - accuracy: 0.9349 - val_loss: 0.2714 - val_accuracy: 0.9473
Epoch 5/5
156/156 [==============================] - 133s 853ms/step - loss: 0.6150 - accuracy: 0.9355 - val_loss: 0.4389 - val_accuracy: 0.9653


For fine tuning I unfreez the backbone, and then finetune the whole model.

In [16]:
model_MobileNetV2.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_MobileNetV2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/MobileNetV2/" + "MobileNetV2_finetune"
tensorboard_callback_MobileNetV2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_MobileNetV2.fit(training_data,
            epochs=25,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_MobileNetV2] 
            )

Epoch 1/25
156/156 [==============================] - 137s 879ms/step - loss: 0.2693 - accuracy: 0.9256 - val_loss: 46.9865 - val_accuracy: 0.5319
Epoch 2/25
156/156 [==============================] - 137s 878ms/step - loss: 0.0968 - accuracy: 0.9632 - val_loss: 13.1798 - val_accuracy: 0.6759
Epoch 3/25
156/156 [==============================] - 135s 867ms/step - loss: 0.0656 - accuracy: 0.9749 - val_loss: 3.1204 - val_accuracy: 0.8662
Epoch 4/25
156/156 [==============================] - 134s 862ms/step - loss: 0.0649 - accuracy: 0.9756 - val_loss: 2.2655 - val_accuracy: 0.8914
Epoch 5/25
156/156 [==============================] - 135s 868ms/step - loss: 0.0589 - accuracy: 0.9782 - val_loss: 0.4449 - val_accuracy: 0.9451
Epoch 6/25
156/156 [==============================] - 135s 867ms/step - loss: 0.0480 - accuracy: 0.9806 - val_loss: 0.3923 - val_accuracy: 0.9609
Epoch 7/25
156/156 [==============================] - 135s 866ms/step - loss: 0.0453 - accuracy: 0.9843 - val_loss: 0.5124

### ResNet50

In [9]:
input_size = (150, 150 ,3)

ResNet50_model = keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=input_size)
ResNet50_model.trainable = False


model_ResNet50 = keras.models.Sequential()
model_ResNet50.add(ResNet50_model)
model_ResNet50.add(Flatten())
model_ResNet50.add(Dense(128, activation=tf.nn.leaky_relu))
model_ResNet50.add(Dropout(0.3))
model_ResNet50.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [10]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_ResNet50.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/ResNet50/" + "ResNet50_transfer"
tensorboard_callback_ResNet50 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_ResNet50.fit(training_data,
            epochs=5,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_ResNet50] 
            )

Epoch 1/5
156/156 [==============================] - 138s 885ms/step - loss: 4.9825 - accuracy: 0.5224 - val_loss: 0.8811 - val_accuracy: 0.5030
Epoch 2/5
156/156 [==============================] - 140s 895ms/step - loss: 0.8481 - accuracy: 0.5532 - val_loss: 0.6743 - val_accuracy: 0.5829
Epoch 3/5
156/156 [==============================] - 139s 889ms/step - loss: 0.7032 - accuracy: 0.5844 - val_loss: 0.6717 - val_accuracy: 0.5719
Epoch 4/5
156/156 [==============================] - 136s 869ms/step - loss: 0.6911 - accuracy: 0.5899 - val_loss: 0.6425 - val_accuracy: 0.6334
Epoch 5/5
156/156 [==============================] - 135s 866ms/step - loss: 0.6688 - accuracy: 0.6081 - val_loss: 0.6968 - val_accuracy: 0.5335


For fine tuning I unfreez the backbone, and then finetune the whole model.

In [11]:
model_ResNet50.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_ResNet50.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/ResNet50/" + "ResNet50_finetune"
tensorboard_callback_ResNet50 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_ResNet50.fit(training_data,
            epochs=30,
            validation_data=validation_data,
            validation_steps=valid_set.shape[0]//batch_size,
            steps_per_epoch=train_set.shape[0]//batch_size,
            callbacks=[tensorboard_callback_ResNet50] 
            )

Epoch 1/30
156/156 [==============================] - 138s 882ms/step - loss: 0.6316 - accuracy: 0.8102 - val_loss: 0.7199 - val_accuracy: 0.5000
Epoch 2/30
156/156 [==============================] - 136s 872ms/step - loss: 0.1531 - accuracy: 0.9375 - val_loss: 0.9190 - val_accuracy: 0.5004
Epoch 3/30
156/156 [==============================] - 135s 864ms/step - loss: 0.1127 - accuracy: 0.9541 - val_loss: 0.7215 - val_accuracy: 0.5000
Epoch 4/30
156/156 [==============================] - 136s 875ms/step - loss: 0.0965 - accuracy: 0.9622 - val_loss: 0.7080 - val_accuracy: 0.5038
Epoch 5/30
156/156 [==============================] - 137s 877ms/step - loss: 0.0856 - accuracy: 0.9677 - val_loss: 0.6612 - val_accuracy: 0.6300
Epoch 6/30
156/156 [==============================] - 136s 873ms/step - loss: 0.0854 - accuracy: 0.9670 - val_loss: 0.4891 - val_accuracy: 0.7706
Epoch 7/30
156/156 [==============================] - 137s 877ms/step - loss: 0.0788 - accuracy: 0.9703 - val_loss: 0.1313 -

### Manual Data Manipulation

I defined three functions. Augmentation is for randomly applying transforms to the input image. It uses 5 different transforms such as random brightness, flip, hue and contrast. Also, a very important step in preprocessing is normalization. Normalization should be applied to both train and validation set. In normalization function I simply resize the image to the desired size and then divide value of each pixel to 255 to mapp the range of pizel value to 0 and 1. Finally, I have a load_data function that loads, augments, normilizes and splits the input data. I only apply augmentation to train set as it helps the model to be more robust to noises and smal variations.

In [4]:

def augmentation (image):
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.1, 0.25)
    return image

def normalization (image):
    image = tf.image.resize(image, [150, 150])
    image = tf.cast(image, tf.float16)
    image = (image / 255.0)
    return image

def load_data(data_dir, split, augment=False, augment_factor=2):
    print('Total number of images: {}'.format(len(os.listdir(data_dir))))

    files, labels = list(), list()
    for file in os.listdir(data_dir):
        files.append(file)
        labels.append(file[:3])
    df = pd.DataFrame({'filename':files, 'label':labels})

    train_set, valid_set = train_test_split(df, test_size=split,random_state=seed)
    print('Number of images in training set: {}'.format(train_set.shape[0]))
    print('Number of images in validation set: {}'.format(valid_set.shape[0]))

    images_valid = []
    labels_valid = []
    # pbar = tqdm(total=len(valid_set))
    # pbar.set_description("Loading Validation Set")
    for index, row in valid_set.iterrows():
        img = cv2.imread(os.path.join(data_dir,row['filename']))
        if img is not None:
            images_valid.append(normalization(img))
            if row['label'] == 'cat':
                labels_valid.append(0)
            else:
                labels_valid.append(1)
            # pbar.update()

    images_train = []
    labels_train = []
    pbar2 = tqdm(total=len(train_set))
    pbar2.set_description("Loading Train Set")
    for index, row in train_set.iterrows():
        img = cv2.imread(os.path.join(data_dir,row['filename']))
        if img is not None:
            images_train.append(normalization(img))
            if row['label'] == 'cat':
                labels_train.append(0)
            else:
                labels_train.append(1)
            pbar2.update()
            if augment:
                for i in range (augment_factor):
                    image = augmentation (normalization(img))
                    images_train.append(image)
                    if row['label'] == 'cat':
                        labels_train.append(0)
                    else:
                        labels_train.append(1)

    return images_train, labels_train, images_valid, labels_valid


In [5]:
images_train, labels_train, images_valid, labels_valid = load_data (data_dir, 0.2, True, 1)

Total number of images: 25000
Number of images in training set: 20000
Number of images in validation set: 5000


Loading Train Set: 100%|█████████▉| 19995/20000 [04:34<00:00, 73.62it/s]

Loading Train Set: 100%|██████████| 20000/20000 [04:50<00:00, 73.62it/s]

In [6]:
images_train_array = np.array(images_train)
labels_train_array = np.array(labels_train)
images_valid_array = np.array(images_valid)
labels_valid_array = np.array(labels_valid)

### VGG16

In [7]:
input_size = (150, 150 ,3)

VGG_model = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_shape=input_size)
VGG_model.trainable = False


model_VGG = keras.models.Sequential()
model_VGG.add(VGG_model)
model_VGG.add(Flatten())
model_VGG.add(Dense(128, activation=tf.nn.leaky_relu))
model_VGG.add(Dropout(0.3))
model_VGG.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_VGG.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/VGG/" + "VGG_transfer_manual"
tensorboard_callback_VGG = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_VGG.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=5,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_VGG] 
            )

Epoch 1/5
1250/1250 [==============================] - 47s 38ms/step - loss: 0.4431 - accuracy: 0.7724 - val_loss: 0.2233 - val_accuracy: 0.9046
Epoch 2/5
1250/1250 [==============================] - 46s 37ms/step - loss: 0.3749 - accuracy: 0.8156 - val_loss: 0.2038 - val_accuracy: 0.9132
Epoch 3/5
1250/1250 [==============================] - 46s 37ms/step - loss: 0.3528 - accuracy: 0.8306 - val_loss: 0.2084 - val_accuracy: 0.9164
Epoch 4/5
1250/1250 [==============================] - 46s 37ms/step - loss: 0.3408 - accuracy: 0.8364 - val_loss: 0.2040 - val_accuracy: 0.9170
Epoch 5/5
1250/1250 [==============================] - 46s 37ms/step - loss: 0.3250 - accuracy: 0.8443 - val_loss: 0.2395 - val_accuracy: 0.9060


For fine tuning I unfreez the backbone, and then finetune the whole model. Also, I use a smaller learning rate since I am training the whole model.

In [12]:
model_VGG.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.0015)
model_VGG.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/VGG/" + "VGG_finetune_manual"
tensorboard_callback_VGG = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_VGG.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=30,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_VGG] 
            )

Epoch 1/30
1250/1250 [==============================] - 110s 88ms/step - loss: 9.1429 - accuracy: 0.5045 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 2/30
1250/1250 [==============================] - 109s 87ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/30
1250/1250 [==============================] - 109s 87ms/step - loss: 0.6931 - accuracy: 0.5061 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/30
1250/1250 [==============================] - 110s 88ms/step - loss: 0.6932 - accuracy: 0.5037 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/30
1250/1250 [==============================] - 109s 88ms/step - loss: 0.6933 - accuracy: 0.4978 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 6/30
1250/1250 [==============================] - 109s 87ms/step - loss: 0.6933 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/30
1250/1250 [==============================] - 109s 87ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0

### MobileNetV2

In [10]:
input_size = (150, 150 ,3)

MobileNetV2_model = keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=input_size)
MobileNetV2_model.trainable = False


model_MobileNetV2 = keras.models.Sequential()
model_MobileNetV2.add(MobileNetV2_model)
model_MobileNetV2.add(Flatten())
model_MobileNetV2.add(Dense(128, activation=tf.nn.leaky_relu))
model_MobileNetV2.add(Dropout(0.3))
model_MobileNetV2.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_MobileNetV2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/MobileNetV2/" + "MobileNetV2_transfer_manual"
tensorboard_callback_MobileNetV2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_MobileNetV2.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=5,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_MobileNetV2] 
            )

Epoch 1/5
1250/1250 [==============================] - 21s 17ms/step - loss: 6.8427 - accuracy: 0.8399 - val_loss: 3.5835 - val_accuracy: 0.9570
Epoch 2/5
1250/1250 [==============================] - 20s 16ms/step - loss: 9.2395 - accuracy: 0.8715 - val_loss: 7.5227 - val_accuracy: 0.9478
Epoch 3/5
1250/1250 [==============================] - 20s 16ms/step - loss: 11.7363 - accuracy: 0.8831 - val_loss: 8.9021 - val_accuracy: 0.9538
Epoch 4/5
1250/1250 [==============================] - 21s 17ms/step - loss: 11.6314 - accuracy: 0.8931 - val_loss: 6.5515 - val_accuracy: 0.9488
Epoch 5/5
1250/1250 [==============================] - 20s 16ms/step - loss: 12.6740 - accuracy: 0.9004 - val_loss: 15.2347 - val_accuracy: 0.9446


For fine tuning I unfreez the backbone, and then finetune the whole model.

In [12]:
model_MobileNetV2.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_MobileNetV2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/MobileNetV2/" + "MobileNetV2_finetune_manual"
tensorboard_callback_MobileNetV2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_MobileNetV2.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=25,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_MobileNetV2] 
            )

Epoch 1/25
1250/1250 [==============================] - 86s 69ms/step - loss: 1.9769 - accuracy: 0.8581 - val_loss: 12.1659 - val_accuracy: 0.8718
Epoch 2/25
1250/1250 [==============================] - 90s 72ms/step - loss: 0.2022 - accuracy: 0.9275 - val_loss: 0.5182 - val_accuracy: 0.9478
Epoch 3/25
1250/1250 [==============================] - 89s 71ms/step - loss: 0.8656 - accuracy: 0.8098 - val_loss: 2.2088 - val_accuracy: 0.7562
Epoch 4/25
1250/1250 [==============================] - 88s 71ms/step - loss: 0.2304 - accuracy: 0.9103 - val_loss: 0.2171 - val_accuracy: 0.9294
Epoch 5/25
1250/1250 [==============================] - 88s 71ms/step - loss: 0.1364 - accuracy: 0.9457 - val_loss: 0.1458 - val_accuracy: 0.9484
Epoch 6/25
1250/1250 [==============================] - 89s 71ms/step - loss: 0.0902 - accuracy: 0.9651 - val_loss: 0.1484 - val_accuracy: 0.9534
Epoch 7/25
1250/1250 [==============================] - 89s 71ms/step - loss: 0.0740 - accuracy: 0.9717 - val_loss: 0.2288 

: 

: 

### ResNet50

In [8]:
input_size = (150, 150 ,3)

ResNet50_model = keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=input_size)
ResNet50_model.trainable = False


model_ResNet50 = keras.models.Sequential()
model_ResNet50.add(ResNet50_model)
model_ResNet50.add(Flatten())
model_ResNet50.add(Dense(128, activation=tf.nn.leaky_relu))
model_ResNet50.add(Dropout(0.3))
model_ResNet50.add(Dense(2, activation='softmax'))

For training the model, I use Adam optimizer since it uses adaptive learning rate and it has shown great stability to the intial parameters. I use categorical crossentropy for loss function since we have a binary classification problem. Since I am only training the classifier (backbone is frozen), few epochs is enough.
I am using tensorboard for logging and saving the history of the training.

In [9]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model_ResNet50.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/ResNet50/" + "ResNet50_transfer_manual"
tensorboard_callback_ResNet50 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_ResNet50.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=5,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_ResNet50] 
            )

Epoch 1/5
1250/1250 [==============================] - 40s 32ms/step - loss: 1.5925 - accuracy: 0.5210 - val_loss: 1.2900 - val_accuracy: 0.5016
Epoch 2/5
1250/1250 [==============================] - 39s 31ms/step - loss: 0.9396 - accuracy: 0.5408 - val_loss: 1.0348 - val_accuracy: 0.5308
Epoch 3/5
1250/1250 [==============================] - 38s 30ms/step - loss: 1.1573 - accuracy: 0.5423 - val_loss: 0.8156 - val_accuracy: 0.5288
Epoch 4/5
1250/1250 [==============================] - 38s 31ms/step - loss: 1.0921 - accuracy: 0.5592 - val_loss: 0.7789 - val_accuracy: 0.5928
Epoch 5/5
1250/1250 [==============================] - 38s 30ms/step - loss: 1.4550 - accuracy: 0.5550 - val_loss: 0.5754 - val_accuracy: 0.7088


For fine tuning I unfreez the backbone, and then finetune the whole model.

In [10]:
model_ResNet50.layers[0].trainable=True

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model_ResNet50.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

log_dir = "logs/ResNet50/" + "ResNet50_finetune_manual"
tensorboard_callback_ResNet50 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model_ResNet50.fit(images_train_array,
            labels_train_array,
            batch_size=32,
            epochs=25,
            validation_data=(images_valid_array, labels_valid_array),
            callbacks=[tensorboard_callback_ResNet50] 
            )

Epoch 1/25
1250/1250 [==============================] - 102s 81ms/step - loss: 0.4609 - accuracy: 0.8325 - val_loss: 0.2045 - val_accuracy: 0.9166
Epoch 2/25
1250/1250 [==============================] - 100s 80ms/step - loss: 0.1999 - accuracy: 0.9213 - val_loss: 0.3725 - val_accuracy: 0.8610
Epoch 3/25
1250/1250 [==============================] - 99s 79ms/step - loss: 0.1443 - accuracy: 0.9424 - val_loss: 3.7035 - val_accuracy: 0.5824
Epoch 4/25
1250/1250 [==============================] - 100s 80ms/step - loss: 1.0788 - accuracy: 0.5871 - val_loss: 0.6403 - val_accuracy: 0.6404
Epoch 5/25
1250/1250 [==============================] - 98s 79ms/step - loss: 0.6737 - accuracy: 0.5934 - val_loss: 0.6036 - val_accuracy: 0.6802
Epoch 6/25
1250/1250 [==============================] - 99s 79ms/step - loss: 0.6311 - accuracy: 0.6409 - val_loss: 0.5232 - val_accuracy: 0.7358
Epoch 7/25
1250/1250 [==============================] - 99s 79ms/step - loss: 0.5674 - accuracy: 0.7028 - val_loss: 0.493

All the results and training process logs are saveed in log directory and using Tensorboard you can check them out. 